In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,37735
2,Huelva,Confirmados PDIA 14 días,2723
3,Huelva,Tasa PDIA 14 días,"530,6233801664166"
4,Huelva,Confirmados PDIA 7 días,1344
5,Huelva,Tasa PDIA 7 dias,"261,9015141181285"
6,Huelva,Total Confirmados,37942
7,Huelva,Curados,33373
8,Huelva,Fallecidos,395


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  37735.0


/tmp/ipykernel_146458/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  10547.0


/tmp/ipykernel_146458/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

/tmp/ipykernel_146458/2690025379.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_146458/2690025379.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_146458/2690025379.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 339 personas en los últimos 7 días 

Un positivo PCR cada 161 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,37735.0,1344.0,2723.0,513170.0,261.901514,530.623380,233.0
Huelva-Costa,22172.0,859.0,1810.0,289548.0,296.669291,625.112244,124.0
Condado-Campiña,11864.0,315.0,604.0,156231.0,201.624518,386.607011,69.0
Huelva (capital),10547.0,424.0,892.0,143837.0,294.778117,620.146416,67.0
Sierra de Huelva-Andévalo Central,3326.0,170.0,307.0,67391.0,252.259204,455.550444,40.0
Isla Cristina,2568.0,76.0,175.0,21393.0,355.256392,818.024587,14.0
Palma del Condado (La),1262.0,23.0,37.0,10801.0,212.943246,342.560874,13.0
Gibraleón,764.0,44.0,72.0,12737.0,345.450263,565.282249,12.0
Moguer,1607.0,60.0,118.0,21867.0,274.386061,539.625920,11.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Encinasola,148.0,30.0,30.0,1310.0,2290.076336,2290.076336,11.0
Beas,286.0,23.0,48.0,4341.0,529.831836,1105.736006,2.0
Cumbres Mayores,74.0,16.0,19.0,1749.0,914.808462,1086.335049,9.0
Santa Ana la Real,16.0,3.0,5.0,475.0,631.578947,1052.631579,0.0
Palos de la Frontera,866.0,42.0,98.0,11742.0,357.690342,834.610799,6.0
Isla Cristina,2568.0,76.0,175.0,21393.0,355.256392,818.024587,14.0
Punta Umbría,1013.0,63.0,122.0,15355.0,410.289808,794.529469,5.0
Nerva,223.0,18.0,38.0,5169.0,348.229832,735.151867,2.0
Ayamonte,1600.0,74.0,154.0,21104.0,350.644428,729.719484,10.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Jabugo,76.0,1.0,7.0,2260.0,44.247788,309.734513,0.0,0.142857
Zarza-Perrunal (La),40.0,1.0,6.0,1253.0,79.808460,478.850758,0.0,0.166667
Niebla,316.0,3.0,11.0,4116.0,72.886297,267.249757,0.0,0.272727
Alosno,211.0,1.0,3.0,3933.0,25.425884,76.277651,1.0,0.333333
Paterna del Campo,134.0,1.0,3.0,3457.0,28.926815,86.780445,0.0,0.333333
Escacena del Campo,161.0,2.0,6.0,2287.0,87.450809,262.352427,0.0,0.333333
Santa Bárbara de Casa,38.0,1.0,3.0,1043.0,95.877277,287.631831,0.0,0.333333
San Bartolomé de la Torre,320.0,2.0,6.0,3761.0,53.177346,159.532039,0.0,0.333333
Valverde del Camino,743.0,22.0,59.0,12750.0,172.549020,462.745098,8.0,0.372881
